In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.searchers.hierarchical_searcher import HierarchicalSearcher
from Qommunity.iterative_searcher import IterativeSearcher

In [2]:
G = nx.powerlaw_cluster_graph(n=100, m=1, p=0.1)
num_reads = 100
version = ""
region = "na-west-1"

### Usage

In [3]:
advantage = AdvantageSampler(
    G, num_reads=num_reads, version=version, region=region, use_clique_embedding=True, elapse_times=True, return_sampleset_metadata=True
)

In [4]:
iterative_searcher = IterativeSearcher(advantage)

In [5]:
iterative_searcher.sampler

In [6]:
from Qommunity.samplers.hierarchical.hierarchical_sampler import (
    HierarchicalSampler,
)
from Qommunity.searchers.hierarchical_searcher import (
    HierarchicalSearcher,
)
import networkx as nx
from time import time
from tqdm import tqdm
import numpy as np
import warnings

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler

SAMPLESET_METADATA_KEYARG = "return_sampleset_metadata"

class MethodArgsWarning(Warning):
    def __init__(self, msg):
        super().__init__(msg)


# Warning format compatible with tqdm
def warn(message, category, filename, lineno, file=None, line=None):
    tqdm.write(f"Warning: {str(message)}")


warnings.showwarning = warn
warnings.simplefilter("always", MethodArgsWarning)


class IterativeHierarchicalSearcher:
    def __init__(self, sampler: HierarchicalSampler) -> None:
        self.sampler = sampler
        self.searcher = HierarchicalSearcher(self.sampler)

    def _default_saving_path(self) -> str:
        return (
            f"{self.sampler.__class__.__name__}"
            + "-network_size_"
            + f"{self.sampler.G.number_of_nodes()}"
        )

    def _verify_kwargs(self, kwargs) -> dict:
        kwargs_unhandled = ["division_tree", "return_modularities"]
        kwargs_warning = []
        for kwarg in kwargs_unhandled:
            if kwarg in kwargs:
                kwargs.pop(kwarg, None)
                kwargs_warning.append(kwarg)
        if kwargs_warning:
            msg = ", ".join(kwargs_warning)
            warnings.warn(
                f"in order to get {msg} run "
                + " IterativeSearcher.run_with_sampleset_info()"
            )

        return kwargs

    def run(
        self,
        num_runs: int,
        save_results: bool = True,
        saving_path: str | None = None,
        elapse_times: bool = True,
        iterative_verbosity: int = 0,
        return_sampleset_metadata: bool = False,
        **kwargs,
    ):
        kwargs = self._verify_kwargs(kwargs)

        if iterative_verbosity >= 1:
            print("Starting community detection iterations")

        if save_results and saving_path is None:
            saving_path = self._default_saving_path()

        modularities = np.zeros((num_runs))
        communities = np.empty((num_runs), dtype=object)
        times = np.zeros((num_runs))
        samplesets_data = np.empty((num_runs), dtype=object)

        for iter in tqdm(range(num_runs)):
            elapsed = time()
            result = self.searcher.hierarchical_community_search(**kwargs)
            times[iter] = time() - elapsed

            if SAMPLESET_METADATA_KEYARG in kwargs:
                result, sampleset_metadata = result

            try:
                modularity_score = nx.community.modularity(
                    self.searcher.sampler.G,
                    result,
                    resolution=self.sampler.resolution,
                )
            except Exception as e:
                print(f"iteration: {iter} exception: {e}")
                modularity_score = -1

            communities[iter] = result
            modularities[iter] = modularity_score
            samplesets_data[iter] = sampleset_metadata

            if save_results:
                np.save(f"{saving_path}_modularities", modularities)
                np.save(f"{saving_path}_communities", communities)
                if elapse_times:
                    np.save(f"{saving_path}_times", times)
                if return_sampleset_metadata:
                    np.save(f"{saving_path}_sampleset_infos", samplesets_data)

            if iterative_verbosity >= 1:
                print(f"Iteration {iter} completed")

        if elapse_times and sampleset_metadata:
            return communities, modularities, times, sampleset_metadata
        if elapse_times:
            return communities, modularities, times
        if return_sampleset_metadata:
            return communities, modularities, samplesets_data
        return communities, modularities

    def run_with_sampleset_info(
        self,
        num_runs: int,
        save_results: bool = True,
        saving_path: str | None = None,
        iterative_verbosity: int = 0,
        return_sampleset_metadata: bool = True,
        **kwargs,
    ):

        if iterative_verbosity >= 1:
            print("Starting community detection iterations")

        if save_results and saving_path is None:
            saving_path = self._default_saving_path()

        modularities = np.zeros((num_runs))
        communities = np.empty((num_runs), dtype=object)
        times = np.zeros((num_runs))
        division_modularities = np.empty((num_runs), dtype=object)
        division_trees = np.empty((num_runs), dtype=object)
        samplesets_data = np.empty((num_runs), dtype=object)

        if return_sampleset_metadata:
            kwargs[SAMPLESET_METADATA_KEYARG] = True

        for iter in tqdm(range(num_runs)):
            elapsed = time()
            result = self.searcher.hierarchical_community_search(
                return_modularities=True,
                division_tree=True,
                **kwargs,
            )
            # Currently only AdvantageSampler among the hierarchical solvers
            # provides sampleset metadata.
            print(isinstance(self.sampler, AdvantageSampler))
            print(return_sampleset_metadata)
            if isinstance(self.sampler, AdvantageSampler) and return_sampleset_metadata:
                (
                    communities_result,
                    div_tree,
                    div_modularities,
                    sampleset_data,
                ) = result
            else:
                (
                    communities_result,
                    div_tree,
                    div_modularities,
                ) = result
            times[iter] = time() - elapsed
            division_trees[iter] = div_tree
            division_modularities[iter] = div_modularities
            samplesets_data[iter] = sampleset_data

            try:
                modularity_score = nx.community.modularity(
                    self.searcher.sampler.G,
                    communities_result,
                    resolution=self.sampler.resolution,
                )
            except Exception as e:
                print(f"iteration: {iter} exception: {e}")
                modularity_score = -1

            communities[iter] = communities_result
            modularities[iter] = modularity_score

            if save_results:
                np.save(f"{saving_path}_modularities", modularities)
                np.save(f"{saving_path}_communities", communities)
                np.save(f"{saving_path}_times", times)
                np.save(f"{saving_path}_division_trees", division_trees)
                np.save(
                    f"{saving_path}_division_modularities",
                    division_modularities,
                )
                np.save(f"{saving_path}_samplesets_data", samplesets_data)

            if iterative_verbosity >= 1:
                print(f"Iteration {iter} completed")

        dtypes = [
            ("communities", object),
            ("modularity", np.float_),
            ("time", np.float_),
            ("division_tree", object),
            ("division_modularities", object),
        ]
        sampleset_components = [
            communities,
            modularities,
            times,
            division_trees,
            division_modularities,
        ]

        if return_sampleset_metadata:
            dtypes.append(("samplesets_data", object))
            sampleset_components.append(samplesets_data)

        sampleset = np.rec.fromarrays(
            sampleset_components,
            dtype=dtypes,
        )

        if not return_sampleset_metadata:
            return sampleset

        results_processed = self._process_results(sampleset)

        return results_processed

    def _process_results(self, sampleset):
        dtype = [si.dwave_sampleset_metadata for si in sampleset[0].samplesets_data][
            0
        ].dtype.descr
        dwave_sampleset_metadata = np.array(
            [
                np.concatenate(
                    [
                        np.array([r], dtype=dtype)
                        for r in [
                            si.dwave_sampleset_info
                            for si in sampleset[run].samplesets_data
                        ]
                    ]
                ).view(np.recarray)
                for run in range(len(sampleset))
            ],
            dtype=object,
        )

        dtype = [si.time_measurements for si in sampleset[0].samplesets_data][
            0
        ].dtype.descr
        time_measurements = np.array(
            [
                np.concatenate(
                    [
                        np.array([r], dtype=dtype)
                        for r in [
                            si.time_measurements
                            for si in sampleset[run].samplesets_data
                        ]
                    ]
                ).view(np.recarray)
                for run in range(len(sampleset))
            ],
            dtype=object,
        )

        results_procesed_dtypes = sampleset.dtype.descr
        results_procesed_dtypes.pop()
        results_procesed_dtypes.append(("dwave_sampleset_metadata", object))
        results_procesed_dtypes.append(("time_measurements", object))
        results_procesed_dtypes

        results_processed_componenets = [
            sampleset.communities,
            sampleset.modularity,
            sampleset.time,
            sampleset.division_tree,
            sampleset.division_modularities,
            dwave_sampleset_metadata,
            time_measurements,
        ]

        results_processed = np.rec.fromarrays(
            results_processed_componenets,
            dtype=results_procesed_dtypes,
        )

        return results_processed


In [7]:
isinstance(iterative_searcher.sampler, AdvantageSampler)

True

In [8]:
res = iterative_searcher.run_with_sampleset_info(
    num_runs=1,
    save_results=True,
    saving_path="advantage_clique_samples",
    iterative_verbosity=1,
    return_sampleset_metadata=True,
)

Starting community detection iterations


100%|██████████| 1/1 [01:44<00:00, 104.67s/it]

True
True
Iteration 0 completed


AttributeError: 'SamplesetData' object has no attribute 'dwave_sampleset_info'

In [ ]:
res[0].dwave_sampleset_infos.qpu_access_time.sum()

697037.52

In [ ]:
for run in res:
    communities = run.communities
    modularities = run.modularity
    div_trees = run.division_tree
    div_modularities = run.division_modularities
    dwave_sampleset_info = run.dwave_sampleset_infos
    times_measured = run.time_measurements

First run

In [ ]:
res[0].time_measurements.find_clique_embedding_time

array([0.1414337, 0.3123298, 0.1391013, 0.3731998, 0.2271785, 0.2742287,
       0.1232225, 0.165031 , 0.1379719, 0.1458408, 0.1343016, 0.1420312,
       0.332811 , 0.100343 , 0.1122439, 0.2090429, 0.1224837, 0.123875 ,
       0.1597369, 0.1049684, 0.1861382, 0.1947471])

Total time of "find_clique_embedding":

In [ ]:
res[0].time_measurements.find_clique_embedding_time.sum()

3.962260900036199

In [ ]:
res[1].dwave_sampleset_infos.qpu_access_time

array([39833.56, 29476.36, 35895.16, 35666.76, 24015.16, 36644.76,
       28392.36, 35576.76, 28437.16, 28173.16, 29365.96, 36489.16,
       28173.56, 36057.16, 26389.96, 35666.76, 35894.76, 28421.16,
       28173.96, 24016.36, 22365.16, 22365.16, 28129.56, 28436.76])

### Heuristic

In [ ]:
advantage = AdvantageSampler(
    G, num_reads=num_reads, version=version, region=region, use_clique_embedding=False, measure_times=True, return_sampleset_info=True
)

In [ ]:
iterative_searcher = IterativeSearcher(advantage)

In [ ]:
res_heu = iterative_searcher.run_with_sampleset_info(
    num_runs=2,
    save_results=True,
    saving_path="heuristic)advantage_clique_samples",
    iterative_verbosity=1,
    return_sampleset_info=True,
)

Starting community detection iterations


 50%|█████     | 1/2 [04:56<04:56, 296.86s/it]

Iteration 0 completed


100%|██████████| 2/2 [14:59<00:00, 449.94s/it]

Iteration 1 completed


In [ ]:
res_heu[0].time_measurements.embedding_composite_time

array([0.2807632, 0.2406278, 0.3359402, 0.302822 , 0.1841993, 0.1517898,
       0.2029491, 0.320833 , 0.2347661, 0.1915635, 0.2737376, 0.2755818,
       0.1963269, 0.2202596, 0.3618938, 0.4035531, 0.4405374, 0.427501 ,
       0.3805251, 0.3727842, 0.3873692, 0.456362 , 0.4334511, 0.3993097])

In [ ]:
res_heu[0].time_measurements

rec.array([(0.2807632, 59.6475311), (0.2406278, 16.8091243),
           (0.3359402,  7.4627778), (0.302822 ,  0.6165701),
           (0.1841993,  0.7094622), (0.1517898,  0.0663413),
           (0.2029491,  0.2232563), (0.320833 ,  0.1497798),
           (0.2347661,  0.1103587), (0.1915635,  0.7009316),
           (0.2737376,  0.2508542), (0.2755818,  0.1312484),
           (0.1963269,  0.0945934), (0.2202596, 38.5239291),
           (0.3618938,  4.9918162), (0.4035531,  0.2753941),
           (0.4405374,  2.4519983), (0.427501 ,  0.1697537),
           (0.3805251,  1.2791192), (0.3727842,  1.8661805),
           (0.3873692,  0.2791576), (0.456362 ,  0.1886594),
           (0.4334511,  0.1984597), (0.3993097,  0.5174587)],
          dtype=[('embedding_composite_time', '<f8'), ('sample_time', '<f8')])

In [ ]:
res_heu[0].time_measurements.embedding_composite_time.sum()

7.4754465000587516

In [ ]:
res_heu[0].time_measurements.sample_time.sum()

137.71475569996983

In [ ]:
res_heu[0].time_measurements.sample_time

array([59.6475311, 16.8091243,  7.4627778,  0.6165701,  0.7094622,
        0.0663413,  0.2232563,  0.1497798,  0.1103587,  0.7009316,
        0.2508542,  0.1312484,  0.0945934, 38.5239291,  4.9918162,
        0.2753941,  2.4519983,  0.1697537,  1.2791192,  1.8661805,
        0.2791576,  0.1886594,  0.1984597,  0.5174587])

In [81]:
res_heu[0].time_measurements.embedding_composite_time

array([0.2807632, 0.2406278, 0.3359402, 0.302822 , 0.1841993, 0.1517898,
       0.2029491, 0.320833 , 0.2347661, 0.1915635, 0.2737376, 0.2755818,
       0.1963269, 0.2202596, 0.3618938, 0.4035531, 0.4405374, 0.427501 ,
       0.3805251, 0.3727842, 0.3873692, 0.456362 , 0.4334511, 0.3993097])